# 机器翻译
- 编码器--解码器
- 注意力机制
- 主要步骤
    - 1 读取和预处理数据
    - 2 注意力机制的编码器解码器(模型)
    - 3 训练模型
    - 4 预测不定长序列
    - 5 评价翻译结果
    
## 1 读取和预处理数据
- 我们先定义一些特殊符号。其中$<pad>$（padding）符号用来添加在较短序列后，直到每个序列等长，而$<bos>$和$<eos>$符号分别表示序列的开始和结束。

In [1]:
import collections
import os
import io
import math
import torch
from torch import nn
import torch.nn.functional as F
import torchtext.vocab as Vocab
import torch.utils.data as Data

import sys
sys.path.append("..") 
# import d2lzh_pytorch as d2l

PAD, BOS, EOS = '<pad>', '<bos>', '<eos>'
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

### 辅助函数预处理数据

In [2]:
# 将一个序列中所有的词记录在all_tokens中以便之后构造词典，然后在该序列后面添加PAD直到序列
# 长度变为max_seq_len，然后将序列保存在all_seqs中
def process_one_seq(seq_tokens, all_tokens, all_seqs, max_seq_len):
    all_tokens.extend(seq_tokens)
    seq_tokens += [EOS] + [PAD] * (max_seq_len - len(seq_tokens) - 1)
    all_seqs.append(seq_tokens)

# 使用所有的词来构造词典。并将所有序列中的词变换为词索引后构造Tensor
def build_data(all_tokens, all_seqs):
    vocab = Vocab.Vocab(collections.Counter(all_tokens),
                        specials=[PAD, BOS, EOS])
    indices = [[vocab.stoi[w] for w in seq] for seq in all_seqs]
    return vocab, torch.tensor(indices)

### 读取数据/构建词典
- 为了演示方便，我们在这里使用一个很小的法语—英语数据集。在这个数据集里，每一行是一对法语句子和它对应的英语句子，中间使用'\t'隔开。在读取数据时，我们在句末附上$<eos>$符号，并可能通过添加$<pad>$符号使每个序列的长度均为**max_seq_len**。我们为法语词和英语词分别创建词典。法语词的索引和英语词的索引相互独立。

In [3]:
def read_data(max_seq_len):
    # in和out分别是input和output的缩写
    in_tokens, out_tokens, in_seqs, out_seqs = [], [], [], []
    with io.open('data/fr-en-small.txt') as f:
        lines = f.readlines()
    for line in lines:
        in_seq, out_seq = line.rstrip().split('\t')
        in_seq_tokens, out_seq_tokens = in_seq.split(' '), out_seq.split(' ')
        if max(len(in_seq_tokens), len(out_seq_tokens)) > max_seq_len - 1:
            continue  # 如果加上EOS后长于max_seq_len，则忽略掉此样本
        process_one_seq(in_seq_tokens, in_tokens, in_seqs, max_seq_len)
        process_one_seq(out_seq_tokens, out_tokens, out_seqs, max_seq_len)
    in_vocab, in_data = build_data(in_tokens, in_seqs)
    out_vocab, out_data = build_data(out_tokens, out_seqs)
    return in_vocab, out_vocab, Data.TensorDataset(in_data, out_data)

In [4]:
# 将序列的最大长度设成7，然后查看读取到的第一个样本。该样本分别包含法语词索引序列和英语词索引序列。
max_seq_len = 7
in_vocab, out_vocab, dataset = read_data(max_seq_len)
dataset[0]

(tensor([ 5,  4, 45,  3,  2,  0,  0]), tensor([ 8,  4, 27,  3,  2,  0,  0]))

## 2 模型
- 使用含注意力机制的编码器—解码器来将一段简短的法语翻译成英语。下面我们来介绍模型的实现。
### 编码器
- 在编码器中，我们将输入语言的词索引通过词嵌入层得到词的表征，然后输入到一个多层门控循环单元中。正如我们在6.5节（循环神经网络的简洁实现）中提到的，PyTorch的nn.GRU实例在前向计算后也会分别返回输出和最终时间步的多层隐藏状态。其中的输出指的是最后一层的隐藏层在各个时间步的隐藏状态，并不涉及输出层计算。注意力机制将这些输出作为键项和值项。

In [5]:
class Encoder(nn.Module):
    def __init__(self, vocab_size, embed_size, num_hiddens, num_layers,
                 drop_prob=0, **kwargs):
        super(Encoder, self).__init__(**kwargs)
        self.embedding = nn.Embedding(vocab_size, embed_size)
        self.rnn = nn.GRU(embed_size, num_hiddens, num_layers, dropout=drop_prob)

    def forward(self, inputs, state):
        # 输入形状是(批量大小, 时间步数)。将输出互换样本维和时间步维
        embedding = self.embedding(inputs.long()).permute(1, 0, 2) # (seq_len, batch, input_size)
        return self.rnn(embedding, state)

    def begin_state(self):
        return None # 隐藏态初始化为None时PyTorch会自动初始化为0

### 
---
- 下面我们来创建一个批量大小为4、时间步数为7的小批量序列输入。设门控循环单元的隐藏层个数为2，隐藏单元个数为16。编码器对该输入执行前向计算后返回的输出形状为(时间步数, 批量大小, 隐藏单元个数)。门控循环单元在最终时间步的多层隐藏状态的形状为(隐藏层个数, 批量大小, 隐藏单元个数)。对于门控循环单元来说，state就是一个元素，即隐藏状态；如果使用长短期记忆，state是一个元组，包含两个元素即隐藏状态和记忆细胞。


In [6]:
encoder = Encoder(vocab_size=10, embed_size=8, num_hiddens=16, num_layers=2)
output, state = encoder(torch.zeros((4, 7)), encoder.begin_state())
output.shape, state.shape # GRU的state是h, 而LSTM的是一个元组(h, c)

(torch.Size([7, 4, 16]), torch.Size([2, 4, 16]))

### 注意力机制
- 我们将实现注意力机制.定义的函数aa：将输入连结后通过含单隐藏层的多层感知机变换。其中隐藏层的输入是解码器的隐藏状态与编码器在所有时间步上隐藏状态的一一连结，且使用tanh函数作为激活函数。输出层的输出个数为1。两个Linear实例均不使用偏差。其中函数aa定义里向量vv的长度是一个超参数，即attention_size。

In [7]:
def attention_model(input_size, attention_size):
    model = nn.Sequential(nn.Linear(input_size, 
                                    attention_size, bias=False),
                          nn.Tanh(),
                          nn.Linear(attention_size, 1, bias=False))
    return model

- 注意力机制的输入包括查询项、键项和值项。设编码器和解码器的隐藏单元个数相同。这里的查询项为解码器在上一时间步的隐藏状态，形状为(批量大小, 隐藏单元个数)；键项和值项均为编码器在所有时间步的隐藏状态，形状为(时间步数, 批量大小, 隐藏单元个数)。注意力机制返回当前时间步的背景变量，形状为(批量大小, 隐藏单元个数)。

In [8]:
def attention_forward(model, enc_states, dec_state):
    """
    enc_states: (时间步数, 批量大小, 隐藏单元个数)
    dec_state: (批量大小, 隐藏单元个数)
    """
    # 将解码器隐藏状态广播到和编码器隐藏状态形状相同后进行连结
    dec_states = dec_state.unsqueeze(dim=0).expand_as(enc_states)
    enc_and_dec_states = torch.cat((enc_states, dec_states), dim=2)
    e = model(enc_and_dec_states)  # 形状为(时间步数, 批量大小, 1)
    alpha = F.softmax(e, dim=0)  # 在时间步维度做softmax运算
    return (alpha * enc_states).sum(dim=0)  # 返回背景变量

- 在下面的例子中，编码器的时间步数为10，批量大小为4，编码器和解码器的隐藏单元个数均为8。注意力机制返回一个小批量的背景向量，每个背景向量的长度等于编码器的隐藏单元个数。因此输出的形状为(4, 8)。

In [9]:
seq_len, batch_size, num_hiddens = 10, 4, 8
model = attention_model(2*num_hiddens, 10) 
enc_states = torch.zeros((seq_len, batch_size, num_hiddens))
dec_state = torch.zeros((batch_size, num_hiddens))
attention_forward(model, enc_states, dec_state).shape # torch.Size([4, 8])

torch.Size([4, 8])

### 含注意力机制的解码器
- 我们直接将编码器在最终时间步的隐藏状态作为解码器的初始隐藏状态。这要求编码器和解码器的循环神经网络使用相同的隐藏层个数和隐藏单元个数。

- 在解码器的前向计算中，我们先通过刚刚介绍的注意力机制计算得到当前时间步的背景向量。由于解码器的输入来自输出语言的词索引，我们将输入通过词嵌入层得到表征，然后和背景向量在特征维连结。我们将连结后的结果与上一时间步的隐藏状态通过门控循环单元计算出当前时间步的输出与隐藏状态。最后，我们将输出通过全连接层变换为有关各个输出词的预测，形状为(批量大小, 输出词典大小)。

In [10]:
class Decoder(nn.Module):
    def __init__(self, vocab_size, embed_size, num_hiddens, num_layers,
                 attention_size, drop_prob=0):
        super(Decoder, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embed_size)
        self.attention = attention_model(2*num_hiddens, attention_size)
        # GRU的输入包含attention输出的c和实际输入, 所以尺寸是 num_hiddens+embed_size
        self.rnn = nn.GRU(num_hiddens + embed_size, num_hiddens, 
                          num_layers, dropout=drop_prob)
        self.out = nn.Linear(num_hiddens, vocab_size)

    def forward(self, cur_input, state, enc_states):
        """
        cur_input shape: (batch, )
        state shape: (num_layers, batch, num_hiddens)
        """
        # 使用注意力机制计算背景向量
        c = attention_forward(self.attention, enc_states, state[-1])
        # 将嵌入后的输入和背景向量在特征维连结, (批量大小, num_hiddens+embed_size)
        input_and_c = torch.cat((self.embedding(cur_input), c), dim=1)
        # 为输入和背景向量的连结增加时间步维，时间步个数为1
        output, state = self.rnn(input_and_c.unsqueeze(0), state)
        # 移除时间步维，输出形状为(批量大小, 输出词典大小)
        output = self.out(output).squeeze(dim=0)
        return output, state

    def begin_state(self, enc_state):
        # 直接将编码器最终时间步的隐藏状态作为解码器的初始隐藏状态
        return enc_state

## 3 训练模型
- 我们先实现batch_loss函数计算一个小批量的损失。解码器在最初时间步的输入是特殊字符BOS。之后，解码器在某时间步的输入为样本输出序列在上一时间步的词，即强制教学。此外，同10.3节（word2vec的实现）中的实现一样，我们在这里也使用掩码变量避免填充项对损失函数计算的影响。
```
elle est vieille .	she is old .
elle est tranquille .	she is quiet .
elle a tort .	she is wrong .
elle est canadienne .	she is canadian .
elle est japonaise .	she is japanese .
ils sont russes .	they are russian .
ils se disputent .	they are arguing .
ils regardent .	they are watching .
ils sont acteurs .	they are actors .
elles sont crevees .	they are exhausted .
il est mon genre !	he is my type !
il a des ennuis .	he is in trouble .
c est mon frere .	he is my brother .
c est mon oncle .	he is my uncle .
il a environ mon age .	he is about my age .
elles sont toutes deux bonnes .	they are both good .
elle est bonne nageuse .	she is a good swimmer .
c est une personne adorable .	he is a lovable person .
il fait du velo .	he is riding a bicycle .
ils sont de grands amis .	they are great friends .
```

In [22]:
def batch_loss(encoder, decoder, X, Y, loss):
    batch_size = X.shape[0]
    enc_state = encoder.begin_state()
    enc_outputs, enc_state = encoder(X, enc_state)
    # 初始化解码器的隐藏状态
    dec_state = decoder.begin_state(enc_state)
    # 解码器在最初时间步的输入是BOS
    dec_input = torch.tensor([out_vocab.stoi[BOS]] * batch_size)
    # 我们将使用掩码变量mask来忽略掉标签为填充项PAD的损失
    mask, num_not_pad_tokens = torch.ones(batch_size,), 0
    l = torch.tensor([0.0])
    i = 0
    for y in Y.permute(1,0): # Y shape: (batch, seq_len)
        print("序列第 [{}]个单词".format(i))
        print("解码器输入dec_input:", dec_input, )
        dec_output, dec_state = decoder(dec_input, dec_state, enc_outputs)
        #print("dec_output:", dec_output, dec_output.size())
        print("dec_state:", dec_state, dec_state.size())
        print("enc_outputs:", enc_outputs)
        #print("y", y)
        #print("CrossEntropyLoss:", loss(dec_output, y))
        print("mask", mask)
        print("-"*30)
        l = l + (mask * loss(dec_output, y)).sum()
        dec_input = y  # 使用强制教学
        num_not_pad_tokens += mask.sum().item()
        # EOS后面全是PAD. 下面一行保证一旦遇到EOS接下来的循环中mask就一直是0
        mask = mask * (y != out_vocab.stoi[EOS]).float()
        i += 1
    return l / num_not_pad_tokens

- 在训练函数中，我们需要同时迭代编码器和解码器的模型参数。

In [23]:
def train(encoder, decoder, dataset, lr, batch_size, num_epochs):
    enc_optimizer = torch.optim.Adam(encoder.parameters(), lr=lr)
    dec_optimizer = torch.optim.Adam(decoder.parameters(), lr=lr)

    loss = nn.CrossEntropyLoss(reduction='none')
    data_iter = Data.DataLoader(dataset, batch_size, shuffle=True)
    for epoch in range(num_epochs):
        l_sum = 0.0
        for X, Y in data_iter:
            print(X)
            print(Y)
            enc_optimizer.zero_grad()
            dec_optimizer.zero_grad()
            l = batch_loss(encoder, decoder, X, Y, loss)
            l.backward()
            enc_optimizer.step()
            dec_optimizer.step()
            l_sum += l.item()
        break
        if (epoch + 1) % 10 == 0:
            print("epoch %d, loss %.3f" % (epoch + 1, l_sum / len(data_iter)))

- 创建模型实例并设置超参数。然后，我们就可以训练模型了。

In [24]:
embed_size, num_hiddens, num_layers = 64, 64, 1
attention_size, drop_prob, lr, batch_size, num_epochs = 10, 0.5, 0.01, 2, 50
encoder = Encoder(len(in_vocab), embed_size, num_hiddens, num_layers,
                  drop_prob)
decoder = Decoder(len(out_vocab), embed_size, num_hiddens, num_layers,
                  attention_size, drop_prob)
train(encoder, decoder, dataset, lr, batch_size, num_epochs)

/usr/local/anaconda2/envs/pt-tf-env/lib/python3.6/site-packages/torch/nn/modules/rnn.py:54: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.5 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


tensor([[ 6,  7, 38,  3,  2,  0,  0],
        [12,  7, 21,  3,  2,  0,  0]])
tensor([[ 7,  5, 31,  3,  2,  0,  0],
        [ 7,  5, 21,  3,  2,  0,  0]])
序列第 [0]个单词
解码器输入dec_input: tensor([1, 1])
dec_state: tensor([[[-0.1393,  0.2936, -0.5886, -0.0687,  0.0963, -0.5384, -0.6605,
           0.0597, -0.2344, -0.1347,  0.0796,  0.2670,  0.4116,  0.1627,
           0.4085,  0.0704, -0.1313,  0.0013,  0.3381, -0.2618,  0.0287,
           0.3162,  0.0669, -0.0386,  0.0549,  0.1877, -0.1045, -0.2432,
          -0.1555, -0.1253, -0.1819, -0.5526,  0.4872,  0.0308,  0.2886,
          -0.0971,  0.6693, -0.0417,  0.0676,  0.0169,  0.3350, -0.8023,
          -0.3994, -0.4915,  0.0326, -0.1598,  0.5736, -0.1902, -0.1287,
          -0.0258, -0.3902,  0.2112,  0.1344,  0.0132, -0.0401,  0.2563,
          -0.2707,  0.0953, -0.3079,  0.0300, -0.1933,  0.0521, -0.3383,
          -0.3936],
         [-0.0659,  0.3224, -0.5626, -0.0946,  0.1006, -0.5432, -0.6270,
           0.0288, -0.1833, -0.1587,  0.022

enc_outputs: tensor([[[-3.1687e-01, -1.3320e-01, -3.0486e-01, -6.2633e-02, -5.2539e-02,
           1.0795e-01,  1.1117e-01,  1.0526e-01, -1.9621e-01, -1.7305e-01,
           5.2908e-02, -1.1708e-01, -3.1992e-01,  2.5680e-01,  1.3125e-01,
          -5.8317e-01,  2.9118e-01,  9.6432e-02, -1.2121e-01, -5.2562e-01,
          -4.9280e-01,  9.9200e-04,  1.0100e-01, -1.4418e-01, -9.7080e-04,
           4.9621e-02, -7.5445e-02,  8.9830e-02,  8.4073e-02, -1.0900e-01,
          -1.8147e-01, -1.2375e-01, -5.9085e-02, -3.4106e-01,  6.5989e-02,
          -1.1115e-01, -1.8222e-01,  2.0451e-02, -9.9988e-02, -3.5648e-01,
           1.4316e-01,  5.8419e-01,  5.7575e-02,  7.9051e-02, -4.5552e-01,
          -2.5276e-01,  3.1515e-01, -7.7021e-02, -2.6506e-02,  3.6586e-01,
           7.0843e-02, -4.8112e-01, -2.0606e-01, -2.6560e-01, -4.0159e-01,
          -2.8730e-02, -3.3728e-01,  1.7582e-01,  2.9807e-02, -3.0381e-01,
           3.6682e-01,  2.1646e-01,  2.1079e-01, -2.2197e-01],
         [-5.9364e-02,  

enc_outputs: tensor([[[-0.2890,  0.1556, -0.2685,  0.0012, -0.0716,  0.0172, -0.1474,
           0.1517, -0.1082, -0.1535, -0.1945, -0.1597, -0.4724,  0.3325,
           0.1116, -0.5082,  0.3477,  0.2603,  0.0068, -0.6025, -0.5775,
          -0.0787,  0.1094, -0.0924,  0.0407,  0.0145,  0.0717,  0.2579,
          -0.0661, -0.1149,  0.0403, -0.0960, -0.0252, -0.3570, -0.1025,
          -0.1044, -0.1947,  0.1145, -0.0769, -0.3539,  0.1510,  0.6411,
          -0.0553,  0.0845, -0.6039, -0.3417,  0.3798, -0.1020,  0.0173,
           0.3886,  0.1477, -0.3305, -0.2986, -0.4481, -0.4462,  0.1228,
          -0.3850,  0.1912,  0.0674, -0.2659,  0.2090,  0.1939,  0.2671,
          -0.3651],
         [-0.0710,  0.3148,  0.1467,  0.3958, -0.3162,  0.1582,  0.0101,
          -0.0057, -0.0248, -0.2350,  0.3197, -0.1170, -0.5608, -0.0145,
           0.3787, -0.2327,  0.5809, -0.5215, -0.5376, -0.6610, -0.4188,
          -0.2015,  0.5406,  0.2518, -0.4450, -0.4262, -0.0457,  0.5708,
          -0.1106,

enc_outputs: tensor([[[-0.2643,  0.4564, -0.2593,  0.0541, -0.0949, -0.0707, -0.4059,
           0.1712, -0.0359, -0.1571, -0.2287, -0.2139, -0.5895,  0.3679,
           0.1058, -0.4368,  0.4438,  0.3126,  0.1805, -0.6815, -0.6550,
          -0.1472,  0.1178, -0.0509,  0.2158, -0.0236,  0.2572,  0.3204,
          -0.1213, -0.1323,  0.2779, -0.1086,  0.0890, -0.4043, -0.2370,
          -0.0740, -0.2013,  0.2110, -0.0625, -0.3476,  0.1305,  0.7000,
          -0.1356,  0.0533, -0.6974, -0.4555,  0.4434, -0.1698,  0.0568,
           0.4341,  0.1590, -0.1960, -0.4161, -0.5756, -0.4891,  0.2558,
          -0.4767,  0.1833,  0.0714, -0.2048, -0.0357,  0.1653,  0.3216,
          -0.4956],
         [-0.3986,  0.1077, -0.1079,  0.4751,  0.2374,  0.0763,  0.0478,
          -0.6380,  0.3585, -0.1490,  0.4389, -0.1711,  0.0351,  0.3032,
           0.1507,  0.5181,  0.2568,  0.0916,  0.2709,  0.1861, -0.5871,
          -0.1010,  0.2366, -0.1209, -0.5268,  0.0956, -0.4196, -0.2877,
          -0.0054,

       grad_fn=<StackBackward>)
mask tensor([1., 1.])
------------------------------
序列第 [5]个单词
解码器输入dec_input: tensor([3, 2])
dec_state: tensor([[[ 0.9502,  0.9803, -0.9556,  0.9371,  0.7209,  0.4109, -0.6240,
          -0.6508, -0.0789, -0.9733, -0.9201, -0.9665,  0.7747,  0.9893,
          -0.9453,  0.6420,  0.9026, -0.6839, -0.8657,  0.6164,  0.9007,
          -0.6406, -0.6550,  0.9572,  0.9462, -0.9686, -0.5413, -0.9424,
           0.7813, -0.8729, -0.9535,  0.9504,  0.8639,  0.6936, -0.7287,
          -0.8804,  0.8292, -0.7553,  0.6777,  0.8122, -0.9759,  0.7852,
           0.1651, -0.7995, -0.8487,  0.9599, -0.7238,  0.8786, -0.2760,
           0.8642, -0.7897,  0.9662,  0.3951, -0.8005,  0.9013,  0.2070,
           0.9395,  0.7273, -0.6872, -0.4580, -0.9870,  0.9769, -0.2916,
          -0.8772],
         [ 0.8773,  0.9191, -0.9887,  0.8248,  0.6969,  0.6947, -0.9124,
          -0.2113, -0.4510,  0.6187,  0.3257, -0.9483,  0.1856,  0.8217,
          -0.8419, -0.5020,  0.0926, -0

## 4 预测不定长的序列
- 主流有3种方法来生成解码器在每个时间步的输出。这里我们实现最简单的贪婪搜索(greedy search)。

In [17]:
def translate(encoder, decoder, input_seq, max_seq_len):
    in_tokens = input_seq.split(' ')
    in_tokens += [EOS] + [PAD] * (max_seq_len - len(in_tokens) - 1)
    enc_input = torch.tensor([[in_vocab.stoi[tk] for tk in in_tokens]]) # batch=1
    enc_state = encoder.begin_state()
    enc_output, enc_state = encoder(enc_input, enc_state)
    dec_input = torch.tensor([out_vocab.stoi[BOS]])
    dec_state = decoder.begin_state(enc_state)
    output_tokens = []
    for _ in range(max_seq_len):
        dec_output, dec_state = decoder(dec_input, dec_state, enc_output)
        pred = dec_output.argmax(dim=1)
        pred_token = out_vocab.itos[int(pred.item())]
        if pred_token == EOS:  # 当任一时间步搜索出EOS时，输出序列即完成
            break
        else:
            output_tokens.append(pred_token)
            dec_input = pred
    return output_tokens

- 简单测试一下模型。输入法语句子“ils regardent.”，翻译后的英语句子应该是“they are watching.”。

In [19]:
input_seq = 'ils regardent .'
translate(encoder, decoder, input_seq, max_seq_len)

['they', 'are', 'watching', '.']

## 5 评价翻译结果
- bleu

In [20]:
def bleu(pred_tokens, label_tokens, k):
    len_pred, len_label = len(pred_tokens), len(label_tokens)
    score = math.exp(min(0, 1 - len_label / len_pred))
    for n in range(1, k + 1):
        num_matches, label_subs = 0, collections.defaultdict(int)
        for i in range(len_label - n + 1):
            label_subs[''.join(label_tokens[i: i + n])] += 1
        for i in range(len_pred - n + 1):
            if label_subs[''.join(pred_tokens[i: i + n])] > 0:
                num_matches += 1
                label_subs[''.join(pred_tokens[i: i + n])] -= 1
        score *= math.pow(num_matches / (len_pred - n + 1), math.pow(0.5, n))
    return score

In [25]:
# 我爱你 +<BOS>
# 我爱你 + <I>
# 我爱你 + <love>
# 我爱你 + <you>